In [13]:
# export
import os
import re
from code.config import args 
from loguru import logger
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)  # 设置显示数据的最大列数，防止出现省略号…，导致数据显示不全
pd.set_option('expand_frame_repr', False)  # 当列太多时不自动换行

from sklearn.metrics import classification_report, accuracy_score
# from mylib.utils.pickle import PickleWrapper

In [14]:
import sys
sys.path.append('..')
import seaborn as sns
sns.set(font='Arial Unicode MS')  # 解决Seaborn中文显示问题
from mylib.data.processing.eda import describe

# EDA

In [15]:
!ls {args.DATA_DIR}

test1.txt  train_data.txt  val_data.txt


# 不考虑嵌套的情况

In [107]:
with open(os.path.join(args.DATA_DIR, 'train_data.txt'), encoding='utf8') as f:
    lines = f.readlines()

with open(os.path.join(args.DATA_DIR, 'val_data.txt'), encoding='utf8') as f:
    lines_val = f.readlines()

with open(os.path.join(args.DATA_DIR, 'test1.txt'), encoding='utf8') as f:
    lines_test = f.readlines()

lines.extend(lines_val)

len(lines), len(lines_val), len(lines_test), lines[:2]

(20000,
 5000,
 3000,
 ['（5）房室结消融和起搏器植入作为反复发作或难治性心房内折返性心动过速的替代疗法。|||3    7    pro|||9    13    pro|||16    33    dis|||\n',
  '（6）发作一次伴血流动力学损害的室性心动过速（ventriculartachycardia），可接受导管消融者。|||8    21    dis|||23    44    dis|||50    53    pro|||\n'])

In [17]:
cls = 'dis sym pro equ dru ite bod dep mic'.split()
for c in cls:
    print(f'B-{c}')
    print(f'I-{c}')

B-dis
I-dis
B-sym
I-sym
B-pro
I-pro
B-equ
I-equ
B-dru
I-dru
B-ite
I-ite
B-bod
I-bod
B-dep
I-dep
B-mic
I-mic


In [18]:
cls_dic = dict()
for c in cls:
    cls_dic[c] = set()

sents = []
marks_list = []
for line in lines:
    ts = line.split('|||')
#     print(ts)
#     print(ts[0])
    sent = ts[0]
    marks = ['O'] * len(sent)
    for i in range(len(ts)):
        if i == 0:
#             print(ts[i])
            pass
        else:
            try:
                start, end, cat = re.split(r'\s+', ts[i])
                cls_dic[cat].add(sent[int(start): (int(end)+1)])
#                 marks[int(start)] = f'B-{cat}'
#                 for ii in range((int(start)+1), (int(end)+1)):
#                     marks[ii] = f'I-{cat}'
                    
            except:
                pass

In [19]:
for k, v in cls_dic.items():
    print(k, len(v))

dis 8018
sym 10512
pro 3911
equ 560
dru 2046
ite 1488
bod 6356
dep 103
mic 771


# 完全用正则打标提交

In [69]:
cls = 'dis sym pro equ dru ite bod dep mic'.split()

['dis', 'sym', 'pro', 'equ', 'dru', 'ite', 'bod', 'dep', 'mic']

In [94]:
'8' in cls_dic['sym']

False

In [90]:
for k, v in cls_dic.items():
#     if k != 'mic': continue
    df_ = pd.Series(list(v)).to_frame()
    df_['t'] = df_.iloc[:, 0].str.len()
    df_ = df_.sort_values('t', ascending=False)
    df_ = df_[~df_.iloc[:,0].isin(['的发','，75','抑','评','脱','肠','']+'病 大 正常 黄 心 报道 结果 A G B C D E K S R Z H V T 8 21 抗'.split())]
    
    df_.iloc[:,0] = df_.iloc[:,0].map(lambda s: re.sub(r'\+|\*', '', s))
    
    p = '|'.join(df_.iloc[:,0].tolist())
#     print(p)
    for i in range(len(lines_test)):
        if not lines_test[i].endswith('|||'):
            if i == len(lines_test)-1:
                lines_test[i] =  lines_test[i] + '|||'
            else:
                lines_test[i] =  lines_test[i][:-1] + '|||'
        
#         print(lines_test[i])
#         f = re.finditer(r'新生儿疾病筛查|医学技术|新生儿', line)
        f = re.finditer(p, lines_test[i])
        for ii in f:
            print(ii, k)
#             print(ii.span())
            span = ii.span()
            lines_test[i] =  lines_test[i] + f'{span[0]}    {span[1]-1}    {k}|||'
#         print(lines_test[i])
#         break
#     break

<re.Match object; span=(2, 7), match='新生儿疾病'> dis
<re.Match object; span=(22, 27), match='苯丙酮尿症'> dis
<re.Match object; span=(46, 51), match='新生儿疾病'> dis
<re.Match object; span=(68, 73), match='新生儿疾病'> dis
<re.Match object; span=(82, 87), match='新生儿疾病'> dis
<re.Match object; span=(6, 11), match='新生儿疾病'> dis
<re.Match object; span=(67, 72), match='遗传代谢病'> dis
<re.Match object; span=(79, 84), match='新生儿疾病'> dis
<re.Match object; span=(90, 92), match='细菌'> dis
<re.Match object; span=(61, 66), match='遗传代谢病'> dis
<re.Match object; span=(22, 25), match='代谢病'> dis
<re.Match object; span=(40, 45), match='新生儿疾病'> dis
<re.Match object; span=(58, 63), match='新生儿疾病'> dis
<re.Match object; span=(27, 32), match='遗传代谢病'> dis
<re.Match object; span=(37, 42), match='遗传代谢病'> dis
<re.Match object; span=(40, 45), match='遗传代谢病'> dis
<re.Match object; span=(51, 55), match='代谢紊乱'> dis
<re.Match object; span=(59, 63), match='代谢紊乱'> dis
<re.Match object; span=(67, 71), match='代谢紊乱'> dis
<re.Match object; span=

<re.Match object; span=(7, 9), match='失眠'> dis
<re.Match object; span=(10, 12), match='焦虑'> dis
<re.Match object; span=(17, 19), match='头痛'> dis
<re.Match object; span=(26, 28), match='急性'> dis
<re.Match object; span=(28, 33), match='肌张力障碍'> dis
<re.Match object; span=(91, 94), match='OCD'> dis
<re.Match object; span=(29, 31), match='头痛'> dis
<re.Match object; span=(47, 50), match='低血压'> dis
<re.Match object; span=(36, 40), match='抽动障碍'> dis
<re.Match object; span=(41, 45), match='ADHD'> dis
<re.Match object; span=(26, 28), match='头痛'> dis
<re.Match object; span=(24, 28), match='共济失调'> dis
<re.Match object; span=(18, 20), match='TS'> dis
<re.Match object; span=(43, 46), match='，可作'> dis
<re.Match object; span=(49, 53), match='抽动障碍'> dis
<re.Match object; span=(42, 44), match='A型'> dis
<re.Match object; span=(184, 188), match='普萘洛尔'> dis
<re.Match object; span=(275, 279), match='抽动障碍'> dis
<re.Match object; span=(5, 9), match='抽动障碍'> dis
<re.Match object; span=(19, 22), match='精神病'> dis

<re.Match object; span=(27, 33), match='严重心力衰竭'> dis
<re.Match object; span=(37, 43), match='肝肾功能不全'> dis
<re.Match object; span=(52, 57), match='嗜铬细胞瘤'> dis
<re.Match object; span=(58, 64), match='多发性骨髓瘤'> dis
<re.Match object; span=(5, 8), match='碘过敏'> dis
<re.Match object; span=(29, 31), match='肾脏'> dis
<re.Match object; span=(0, 2), match='静脉'> dis
<re.Match object; span=(43, 48), match='先天性畸形'> dis
<re.Match object; span=(49, 51), match='炎症'> dis
<re.Match object; span=(52, 54), match='肿瘤'> dis
<re.Match object; span=(55, 57), match='外伤'> dis
<re.Match object; span=(63, 65), match='血尿'> dis
<re.Match object; span=(15, 22), match='膀胱输尿管反流'> dis
<re.Match object; span=(26, 31), match='后尿道瓣膜'> dis
<re.Match object; span=(4, 11), match='膀胱输尿管反流'> dis
<re.Match object; span=(24, 27), match='输尿管'> dis
<re.Match object; span=(33, 35), match='肾脏'> dis
<re.Match object; span=(57, 59), match='肾脏'> dis
<re.Match object; span=(7, 9), match='静脉'> dis
<re.Match object; span=(43, 46), match='肾肿瘤

<re.Match object; span=(138, 140), match='急性'> dis
<re.Match object; span=(144, 149), match='前葡萄膜炎'> dis
<re.Match object; span=(162, 168), match='强直性脊柱炎'> dis
<re.Match object; span=(173, 175), match='炎症'> dis
<re.Match object; span=(178, 181), match='关节炎'> dis
<re.Match object; span=(182, 184), match='炎症'> dis
<re.Match object; span=(187, 190), match='关节炎'> dis
<re.Match object; span=(197, 200), match='综合征'> dis
<re.Match object; span=(201, 203), match='急性'> dis
<re.Match object; span=(203, 208), match='前葡萄膜炎'> dis
<re.Match object; span=(6, 14), match='幼年特发性关节炎'> dis
<re.Match object; span=(31, 34), match='JIA'> dis
<re.Match object; span=(56, 59), match='关节炎'> dis
<re.Match object; span=(6, 9), match='关节炎'> dis
<re.Match object; span=(58, 60), match='急性'> dis
<re.Match object; span=(61, 63), match='慢性'> dis
<re.Match object; span=(68, 71), match='HLA'> dis
<re.Match object; span=(6, 9), match='JIA'> dis
<re.Match object; span=(9, 12), match='全身型'> dis
<re.Match object; span=(12, 20

<re.Match object; span=(0, 6), match='结核性脑膜炎'> dis
<re.Match object; span=(14, 16), match='闭塞'> dis
<re.Match object; span=(2, 5), match='脑萎缩'> dis
<re.Match object; span=(6, 9), match='脑萎缩'> dis
<re.Match object; span=(38, 41), match='脑萎缩'> dis
<re.Match object; span=(2, 5), match='脑积水'> dis
<re.Match object; span=(8, 14), match='交通性脑积水'> dis
<re.Match object; span=(15, 21), match='阻塞性脑积水'> dis
<re.Match object; span=(22, 28), match='阻塞性脑积水'> dis
<re.Match object; span=(35, 37), match='梗阻'> dis
<re.Match object; span=(43, 48), match='占位性病变'> dis
<re.Match object; span=(2, 5), match='脱髓鞘'> dis
<re.Match object; span=(36, 39), match='脑萎缩'> dis
<re.Match object; span=(2, 9), match='神经皮肤综合征'> dis
<re.Match object; span=(10, 25), match='Sturge-Weber综合征'> dis
<re.Match object; span=(39, 41), match='钙化'> dis
<re.Match object; span=(57, 62), match='结节性硬化'> dis
<re.Match object; span=(6, 8), match='畸形'> dis
<re.Match object; span=(9, 16), match='胼胝体发育不全'> dis
<re.Match object; span=(23, 28), m

...

<re.Match object; span=(36, 40), match='内皮细胞'> bod
<re.Match object; span=(33, 35), match='呕吐'> bod
<re.Match object; span=(36, 38), match='肝区'> bod
<re.Match object; span=(5, 6), match='肝'> bod
<re.Match object; span=(8, 10), match='肝酶'> bod
<re.Match object; span=(7, 8), match='尿'> bod
<re.Match object; span=(15, 16), match='膜'> bod
<re.Match object; span=(17, 19), match='皮肤'> bod
<re.Match object; span=(28, 30), match='肝脏'> bod
<re.Match object; span=(46, 47), match='脾'> bod
<re.Match object; span=(14, 16), match='肝酶'> bod
<re.Match object; span=(24, 26), match='肝脏'> bod
<re.Match object; span=(13, 14), match='肝'> bod
<re.Match object; span=(15, 16), match='肝'> bod
<re.Match object; span=(18, 20), match='肝酶'> bod
<re.Match object; span=(20, 22), match='异常'> bod
<re.Match object; span=(4, 5), match='胆'> bod
<re.Match object; span=(14, 16), match='大便'> bod
<re.Match object; span=(23, 25), match='皮肤'> bod
<re.Match object; span=(28, 29), match='血'> bod
<re.Match object; span=(29, 32), 

<re.Match object; span=(18, 21), match='心瓣膜'> bod
<re.Match object; span=(27, 28), match='硫'> bod
<re.Match object; span=(3, 4), match='硫'> bod
<re.Match object; span=(24, 25), match='盐'> bod
<re.Match object; span=(25, 26), match='水'> bod
<re.Match object; span=(0, 1), match='低'> bod
<re.Match object; span=(13, 15), match='中央'> bod
<re.Match object; span=(1, 2), match='颈'> bod
<re.Match object; span=(2, 4), match='面部'> bod
<re.Match object; span=(5, 7), match='胸部'> bod
<re.Match object; span=(7, 9), match='肿块'> bod
<re.Match object; span=(15, 18), match='支气管'> bod
<re.Match object; span=(20, 22), match='肺部'> bod
<re.Match object; span=(31, 32), match='肺'> bod
<re.Match object; span=(35, 37), match='胸腔'> bod
<re.Match object; span=(46, 48), match='腹部'> bod
<re.Match object; span=(48, 50), match='胃肠'> bod
<re.Match object; span=(58, 60), match='腹部'> bod
<re.Match object; span=(60, 62), match='手术'> bod
<re.Match object; span=(65, 66), match='口'> bod
<re.Match object; span=(70, 72), match

<re.Match object; span=(29, 31), match='感染'> mic
<re.Match object; span=(39, 41), match='结核'> mic
<re.Match object; span=(84, 86), match='RV'> mic
<re.Match object; span=(4, 6), match='感染'> mic
<re.Match object; span=(12, 14), match='感染'> mic
<re.Match object; span=(21, 23), match='感染'> mic
<re.Match object; span=(2, 6), match='肠道病毒'> mic
<re.Match object; span=(12, 14), match='感染'> mic
<re.Match object; span=(17, 22), match='柯萨奇病毒'> mic
<re.Match object; span=(25, 29), match='埃可病毒'> mic
<re.Match object; span=(21, 25), match='肠道病毒'> mic
<re.Match object; span=(25, 27), match='感染'> mic
<re.Match object; span=(578, 580), match='CT'> mic
<re.Match object; span=(641, 643), match='CJ'> mic
<re.Match object; span=(1542, 1544), match='Sa'> mic
<re.Match object; span=(28, 33), match='百日咳杆菌'> mic
<re.Match object; span=(57, 59), match='感染'> mic
<re.Match object; span=(78, 80), match='杆菌'> mic
<re.Match object; span=(83, 88), match='百日咳杆菌'> mic
<re.Match object; span=(93, 95), match='病原'> mic
<

<re.Match object; span=(33, 38), match='百日咳杆菌'> mic
<re.Match object; span=(54, 58), match='细菌感染'> mic
<re.Match object; span=(380, 382), match='Sa'> mic
<re.Match object; span=(1130, 1132), match='Sa'> mic
<re.Match object; span=(1287, 1289), match='Sa'> mic
<re.Match object; span=(5, 7), match='细胞'> mic
<re.Match object; span=(18, 20), match='细胞'> mic
<re.Match object; span=(24, 26), match='细胞'> mic
<re.Match object; span=(36, 38), match='细胞'> mic
<re.Match object; span=(42, 45), match='包涵体'> mic
<re.Match object; span=(20, 22), match='细胞'> mic
<re.Match object; span=(26, 28), match='细胞'> mic
<re.Match object; span=(34, 36), match='感染'> mic
<re.Match object; span=(2, 7), match='巨细胞病毒'> mic
<re.Match object; span=(9, 14), match='巨细胞病毒'> mic
<re.Match object; span=(15, 18), match='CMV'> mic
<re.Match object; span=(19, 21), match='感染'> mic
<re.Match object; span=(31, 36), match='巨细胞病毒'> mic
<re.Match object; span=(5, 7), match='感染'> mic
<re.Match object; span=(12, 15), match='CMV'> mic


<re.Match object; span=(37, 39), match='RV'> mic
<re.Match object; span=(97, 99), match='RV'> mic
<re.Match object; span=(54, 56), match='CT'> mic
<re.Match object; span=(76, 78), match='CT'> mic
<re.Match object; span=(32, 34), match='MP'> mic
<re.Match object; span=(47, 49), match='细胞'> mic
<re.Match object; span=(72, 74), match='细胞'> mic
<re.Match object; span=(64, 66), match='MP'> mic
<re.Match object; span=(2, 4), match='MP'> mic
<re.Match object; span=(0, 2), match='病原'> mic
<re.Match object; span=(4, 10), match='溶血性链球菌'> mic
<re.Match object; span=(10, 12), match='感染'> mic
<re.Match object; span=(4, 6), match='感染'> mic
<re.Match object; span=(64, 66), match='细胞'> mic
<re.Match object; span=(86, 88), match='细胞'> mic
<re.Match object; span=(8, 10), match='细胞'> mic
<re.Match object; span=(17, 19), match='细胞'> mic
<re.Match object; span=(7, 9), match='细胞'> mic
<re.Match object; span=(3, 5), match='细胞'> mic
<re.Match object; span=(22, 24), match='细胞'> mic
<re.Match object; span=(62, 

In [98]:
with open('sub_reg.txt', 'w') as f:
    f.write('\n'.join(lines_test))

In [97]:
lines_test[-1]

'全身麻醉对不合作或检查时间较长者较适用，多以静脉应用丙泊酚为主，复合芬太尼、瑞芬太尼、舒芬太尼之一种，亦有复合氯胺酮的。|||22    23    dis|||0    3    pro|||22    25    pro|||34    36    dru|||39    41    dru|||44    46    dru|||55    57    dru|||0    1    bod|||22    23    bod|||55    55    bod|||'

In [12]:
for line in lines_test:
    print(line)
    break

六、新生儿疾病筛查的发展趋势自1961年开展苯丙酮尿症筛查以来，随着医学技术的发展，符合进行新生儿疾病筛查标准的疾病也在不断增加，无论在新生儿疾病筛查的病种，还是在新生儿疾病筛查的技术方法上，都有了非常显著的进步。



In [50]:
for i in f:
    print(i)
    print(i.span())

<re.Match object; span=(2, 5), match='新生儿'>
(2, 5)
<re.Match object; span=(34, 38), match='医学技术'>
(34, 38)
<re.Match object; span=(46, 49), match='新生儿'>
(46, 49)
<re.Match object; span=(67, 70), match='新生儿'>
(67, 70)
<re.Match object; span=(81, 84), match='新生儿'>
(81, 84)


In [9]:
len(results)

20000

In [10]:
with open(os.path.join(args.DATA_GEN, 'train.txt'), 'w', encoding='utf8') as f:
    f.write('\n\n'.join(results[:]))

In [11]:
with open(os.path.join(args.DATA_GEN, 'dev.txt'), 'w', encoding='utf8') as f:
    f.write('\n\n'.join(results[15000:]))

In [12]:
test = []
for i in lines_test:
    test.append('\n'.join(list(i[:-1])))
    
with open(os.path.join(args.DATA_GEN, 'test.txt'), 'w', encoding='utf8') as f:
    f.write('\n\n'.join(test))

# 模型+正则
思路：如果模型已经标注过的，不变。只改变O的部分

In [143]:
with open(os.path.join(args.DATA_GEN, 'test_predictions_combine.txt'), encoding='utf8') as f:
    lines_test_model = f.readlines()
    
cls = 'dis sym pro equ dru ite bod dep mic'.split()

sents = []
marks_list = []
for line in lines_test_model:
    ts = line.split('|||')
#     print(ts)
#     print(ts[0])
    sent = ts[0]
    marks = ['O'] * len(sent)
    for i in range(len(ts)):
        if i == 0:
#             print(ts[i])
            pass
        else:
            try:
                start, end, cat = re.split(r'\s+', ts[i])
                marks[int(start)] = f'B-{cat}'
                for ii in range((int(start)+1), (int(end)+1)):
                    marks[ii] = f'I-{cat}'
            except:
                pass
    sents.append(sent)
    marks_list.append(marks)

len(sents), len(marks_list)

(3000, 3000)

In [144]:
for k, v in cls_dic.items():
#     if k != 'mic': continue
    if k not in ['dep', 'ite']: continue
    df_ = pd.Series(list(v)).to_frame()
    df_['t'] = df_.iloc[:, 0].str.len()
    df_ = df_[df_.t > 2]
    df_ = df_.sort_values('t', ascending=False)
    df_ = df_[df_.iloc[:,0].isin(['的发','，75','抑','评','脱','肠','','实验']
                                 +'病 大 正常 黄 心 报道 结果'.split()
                                 +'A G B C D E K S R Z H V T 8 21 抗'.split())==False]
    
    df_.iloc[:,0] = df_.iloc[:,0].map(lambda s: re.sub(r'\+|\*', '', s))
    
    p = '|'.join(df_.iloc[:,0].tolist())
#     print(p)
    for i in range(len(lines_test_model)):
#         if not lines_test_model[i].endswith('|||\n'):
#             if i == len(lines_test_model)-1:
#                 lines_test_model[i] =  lines_test_model[i] + '|||'
#             else:
#                 lines_test_model[i] =  lines_test_model[i][:-1] + '|||'
        if lines_test_model[i].endswith('\n'):
            lines_test_model[i] =  lines_test_model[i][:-1]
#         print(lines_test_model[i])
#         f = re.finditer(r'新生儿疾病筛查|医学技术|新生儿', line)
        f = re.finditer(p, lines_test[i])
        for ii in f:
#             print(ii, k)
#             print(ii.span())
            span = ii.span()
            flag_ = 0
            for iii in range(span[0], span[1]):
                if marks_list[i][iii] != 'O':
                    flag_=1
                    break
            if flag_ == 0:
                print(ii, k)
                lines_test_model[i] =  lines_test_model[i] + f'{span[0]}    {span[1]-1}    {k}|||'
#         print(lines_test_model[i])
#         break
#     break

<re.Match object; span=(61, 64), match='Scr'> ite
<re.Match object; span=(85, 88), match='Scr'> ite
<re.Match object; span=(130, 133), match='自身抗'> ite
<re.Match object; span=(9, 12), match='核细胞'> ite
<re.Match object; span=(1, 6), match='实验室检查'> ite
<re.Match object; span=(17, 22), match='血清学检查'> ite
<re.Match object; span=(3, 10), match='特异性抗体检测'> ite
<re.Match object; span=(4, 9), match='实验室检查'> ite
<re.Match object; span=(147, 150), match='脂肪酸'> ite
<re.Match object; span=(51, 54), match='清蛋白'> ite
<re.Match object; span=(77, 80), match='氨基酸'> ite
<re.Match object; span=(34, 37), match='脂肪酸'> ite
<re.Match object; span=(35, 38), match='氨基酸'> ite
<re.Match object; span=(2, 6), match='羊水细胞'> ite
<re.Match object; span=(2, 8), match='自主神经功能'> ite
<re.Match object; span=(16, 20), match='呼气时间'> ite
<re.Match object; span=(3, 7), match='时间常数'> ite
<re.Match object; span=(8, 11), match='顺应性'> ite
<re.Match object; span=(62, 65), match='肺容积'> ite
<re.Match object; span=(29, 32), match='顺应性

<re.Match object; span=(22, 25), match='氧浓度'> ite
<re.Match object; span=(13, 29), match='cmH<sub>2</sub>O'> ite
<re.Match object; span=(17, 21), match='血氧分压'> ite
<re.Match object; span=(54, 59), match='血流动力学'> ite
<re.Match object; span=(3, 6), match='氧浓度'> ite
<re.Match object; span=(13, 28), match='PaO<sub>2</sub>'> ite
<re.Match object; span=(23, 29), match='二氧化碳分压'> ite
<re.Match object; span=(12, 28), match='PaCO<sub>2</sub>'> ite
<re.Match object; span=(39, 54), match='PaO<sub>2</sub>'> ite
<re.Match object; span=(123, 129), match='二氧化碳分压'> ite
<re.Match object; span=(157, 173), match='PaCO<sub>2</sub>'> ite
<re.Match object; span=(31, 47), match='PaCO<sub>2</sub>'> ite
<re.Match object; span=(26, 29), match='平均压'> ite
<re.Match object; span=(11, 14), match='V/Q'> ite
<re.Match object; span=(28, 30), match='Na'> ite
<re.Match object; span=(10, 12), match='Na'> ite
<re.Match object; span=(2, 4), match='Na'> ite
<re.Match object; span=(41, 43), match='Na'> ite
<re.Match object; s

In [145]:
with open('sub_reg_model_dep_ite.txt', 'w') as f:
    f.write('\n'.join(lines_test_model))

In [104]:
len(lines_test_model[i])

201

In [106]:
lines_test_model[i][:107]

'六、新生儿疾病筛查的发展趋势自1961年开展苯丙酮尿症筛查以来，随着医学技术的发展，符合进行新生儿疾病筛查标准的疾病也在不断增加，无论在新生儿疾病筛查的病种，还是在新生儿疾病筛查的技术方法上，都有了非常显著的进步。'

In [137]:
lines_test_model[1]

'近年来，国际新生儿疾病筛查的发展趋势主要注重在以下几个方面：1.串联质谱技术（tandemmassspectrometry）用于新生儿遗传代谢病筛查长期以来，新生儿疾病筛查主要采用细菌抑制法、放射免疫分析法、时间分辨荧光免疫法以及酶免疫分析法等，这些均是“一项实验检测一种疾病”的方法。|||6    12    pro|||32    37    pro|||64    64    dis|||79    85    pro|||90    94    pro|||96    102    pro|||104    112    pro|||115    120    pro|||67    71    dis|||72    73    pro|||'

In [120]:
lines_test_model[1]

'近年来，国际新生儿疾病筛查的发展趋势主要注重在以下几个方面：1.串联质谱技术（tandemmassspectrometry）用于新生儿遗传代谢病筛查长期以来，新生儿疾病筛查主要采用细菌抑制法、放射免疫分析法、时间分辨荧光免疫法以及酶免疫分析法等，这些均是“一项实验检测一种疾病”的方法。|||6    12    pro|||32    37    pro|||64    64    dis|||79    85    pro|||90    94    pro|||96    102    pro|||104    112    pro|||115    120    pro|||\n67    71    dis|||72    73    pro|||130    131    ite|||130    131    dep|||'

In [122]:
lines_test_model[1][67:72]

'遗传代谢病'

In [123]:
lines_test_model[1][72:74]

'筛查'

In [124]:
lines_test_model[1][130:132]

'实验'

In [138]:
lines_test_model[0]

'六、新生儿疾病筛查的发展趋势自1961年开展苯丙酮尿症筛查以来，随着医学技术的发展，符合进行新生儿疾病筛查标准的疾病也在不断增加，无论在新生儿疾病筛查的病种，还是在新生儿疾病筛查的技术方法上，都有了非常显著的进步。|||2    8    pro|||22    28    dis|||46    52    pro|||68    74    pro|||82    88    pro|||'

In [139]:
lines_test_model[2]

'质谱主要是将被测物质分子电离成各种质荷比m/s不同的带电粒子，然后应用电磁学原理，使这些带电粒子按照质荷比大小在空间或时间上产生分离排列成的图谱，通过测定离子峰的强度，以此获得确定化合物的相对分子质量及分子式。|||0    1    pro|||17    17    ite|||50    51    ite|||'

In [112]:
len(lines_test[0])

108

In [113]:
lines_test[0][91]

'术'